In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

df = pd.read_csv("health_study_dataset.csv")
df.columns = (df.columns.str.strip().str.replace(" ", "_").str.lower())

## Beskrivande analys

In [ ]:
print(f"Medelvärde för åldern: {df["age"].mean():.2f} år \nMedian för åldern: {df["age"].median():.2f} år \nLägsta åldern: {df["age"].min()} år \nHögsta åldern: {df["age"].max()} år")
print(f"\nMedelvärde för vikten: {df["weight"].mean():.2f} kg \nMedian för vikten: {df["weight"].median():.2f} kg \nLägsta vikten: {df["weight"].min()} kg \nHögsta vikten: {df["weight"].max()} kg")
print(f"\nMedelvärde för längden: {df["height"].mean():.2f} cm \nMedian för längden: {df["height"].median():.2f} cm \nKortast: {df["height"].min()} cm \nLängst: {df["height"].max()} cm")
print(f"\nMedelvärde för systoliskt bloodtryck: {df["systolic_bp"].mean():.2f} \nMedian för systoliskt bloodtryck: {df["systolic_bp"].median():.2f} \nLägsta systoliska bloodtryck: {df["systolic_bp"].min()} \nHögsta systoliska bloodtryck: {df["systolic_bp"].max()}")
print(f"\nMedelvärde för kolesterol: {df["cholesterol"].mean():.2f} \nMedian för kolesterol: {df["cholesterol"].median():.2f} \nLägst kolesterol: {df["cholesterol"].min()} \nHögst kolesterol: {df["cholesterol"].max()}")

In [ ]:
plt.rcParams["figure.figsize"] = (8, 5)

fig, ax = plt.subplots()
ax.hist(df["systolic_bp"], bins=30, edgecolor='black')
ax.set_title("Fördelning av systoliskt bloddtryck")
ax.set_xlabel("Systoliskt bloodtryck")
ax.set_ylabel("Värde")
ax.grid(axis="y")
plt.tight_layout()

fig, ax = plt.subplots()
df.boxplot(column="weight", by="sex", ax=ax)
ax.set_title("Vikt per kön")
plt.suptitle("")
ax.set_xlabel("Kön")
ax.set_ylabel("Vikt (kg)")

x = df["age"].to_numpy()
y = df["systolic_bp"].to_numpy()

fig, ax = plt.subplots()
ax.scatter(x, y)
ax.set_title("Ålder vs. systoliskt blodtryck")
ax.set_xlabel("Ålder")
ax.set_ylabel("Systoliskt blodtryck")
ax.grid(True)
k, m = np.polyfit(x, y, 1)
ix = np.argsort(x)
ax.plot(x[ix], (k*x + m)[ix], color="black")

fig, ax = plt.subplots()
smoker_counts = df["smoker"].value_counts()
smoker_counts.plot(kind="bar", color=["green", "red"], edgecolor="black", ax=ax)
ax.set_title("Rökare")
ax.set_xlabel("Rökare")
ax.set_ylabel("Antal")
ax.grid(axis="y")
plt.xticks(rotation=0)
plt.tight_layout()

## Simulering kopplat till caset

In [ ]:
disease_proportions = df["disease"].value_counts(normalize=True)
print("Andel som har sjukdom (0 = har inte sjukdom, 1 = har sjukdom):")
print(disease_proportions)

np.random.seed(42)
p_sick = 0.05875
simulated = np.random.binomial(n=1, p=p_sick, size=1000)

print("\nAndel som har sjukdom i stickprovet: 0.05875")
print(f"Simulerad andel sjukdom: {simulated.mean()}")

- Simuleringen visar att om den sanna andelsen sjuka i populationen är 5.9 % så kan ett stickprov på 1000 personer ge en observerad andel på 5.6 % på grund av slumpmässig variation.

## Konfidensintervall för systoliskt blodtryck
- Valde att göra normalapproximation med hjälp av t-fördelning då populationsstandardavvikelse är okänd.
- Konfidensintervallet blir litet då vi har ett stort stickprov med litet spridning (liten standardavvikelse)
- Uträkning med bootstrap gav ett snävare intervall vilket kan tyda på mer precis uträkning men kan också bero på att bootstrapen underskattar osäkerheten. Normalapproximation är mer konservativ.
- Källa: Business Statistics av Jaggia & Kelly (2021)

In [ ]:
def conf_intervall(x, confidence=0.95):
    mean = x.mean()
    std = x.std(ddof=1)
    n = len(x)
    t_stats = stats.t.ppf(1 - (1 - confidence) / 2, df=n - 1)
    margin = t_stats*(std/np.sqrt(n))
    return mean - margin, mean + margin


def ci_mean_bootstrap(x, B=5000, confidence=0.95):   
    x = np.asarray(x, dtype=float)
    n = len(x)
    boots = np.empty(B)
    for b in range(B):
        boot_sample = np.random.choice(x, size=n)
        boots[b] = np.mean(boot_sample)
    lower = np.percentile(boots, 100*confidence / 2)
    upper = np.percentile(boots, 100*(1 - confidence / 2))
    return lower, upper

low, high = conf_intervall(df["systolic_bp"])
low_boot, high_boot = ci_mean_bootstrap(df["systolic_bp"])
mean_systolic_bp = df["systolic_bp"].mean()

print(f"Medelvärde för systoliskt bloodtryck: {mean_systolic_bp:.2f}")
print(f"95% CI med normalapproximation = ({low:.2f}, {high:.2f})")
print(f"95% CI med bootstrap = ({low_boot:.2f}, {high_boot:.2f})")


## Hypotesprövning
- HO: medelvärde för bloodtryck hos rökare = medelvärde för bloodtryck hos icke-rökare
- H1: medelvärde för bloodtryck hos rökare > medelvärde för bloodtryck hos icke-rökare

Vi gör ett ensidigt Welch's t-test då varianserna för våra två grupper inte kan antas lika (equal_var=False).
- Källa: Scipy docs: stats.ttest_ind

Vi räknar ut ett konfidensintervall för skillnaden i medelvärden med hjälp av t-fördelningen.
- Källa: Business Statistics av Jaggia & Kelly (2021)

In [ ]:
from statsmodels.stats.api import DescrStatsW
from statsmodels.stats.api import CompareMeans

bp_smoker = df[df["smoker"] == "Yes"]["systolic_bp"]
bp_no_smoker = df[df["smoker"] == "No"]["systolic_bp"]

t_stat, p_value = stats.ttest_ind(bp_smoker, bp_no_smoker, equal_var=False, alternative="greater")
res = CompareMeans(DescrStatsW(bp_smoker), DescrStatsW(bp_no_smoker))
low_ci_diff, high_ci_diff = res.tconfint_diff(alpha=0.05, usevar="unequal")

summary = f"""
Medelvärde på bloodtryck för rökare: {bp_smoker.mean():.3f}
Medelvärde på bloodtryck för icke-rökare: {bp_no_smoker.mean():.3f}
Differensen mellan medelvärdena: {bp_smoker.mean() - bp_no_smoker.mean():.3f}
Teststatistika = {t_stat:.3f} \np-värde = {p_value:.3f}
CI 95% för skillnaden i gruppernas medelvärden: {low_ci_diff:.3f}, {high_ci_diff:.3f}
"""
print(summary)

### Slutsats 
- p-värdet > 0.05 vilket gör att vi inte kan förkasta nollhypotesen vid signifikansnivå 95%.
- Konfidensintervallet innehåller 0 vilket också visar att vi inte kan förkasta nollhypotesen
- Detta innebär att det inte finns tillräckligt statistikt stöd för att rökare har högre medelblodtryck än icke-rökare

Gör en simulering för att undersöka hur säkert ditt hypotes­test är (t.ex. hur ofta testet hittar en skillnad när den finns → power).

## Power

- Valde mellan NormalIndPower() och TTestIndPower(). Den senare används när varianserna är lika eller kan anses lika. Då vi inte har antagit lika varians tidigare väljs istället NormalIndPower() för att beräkna power baserat på normalapproximation. 

- NormIndPower() kan användas när vi har normalfördelad data eller tillräckligt stora stickprov där t-fördelningen nästan är lika med normalfördelningen. Vi har två stickprov med ca 200 respektive 550 observationer vilket kan anses som tillräckligt stora.

- Källa: Statsmodels docs: stats.power.TTestIndPower och stats.power.NormalIndPower
- Källa: Business Statistics av Jaggia & Kelly (2021)

In [ ]:
from statsmodels.stats.power import NormalIndPower

mean1 = bp_smoker.mean()
mean2 = bp_no_smoker.mean()
sd1, sd2 = bp_smoker.std(ddof=1), bp_no_smoker.std(ddof=1)
d_welch = (mean1 - mean2) / np.sqrt((sd1**2 + sd2**2) / 2)
alpha = 0.05
n_smoker = len(bp_smoker)
n_no_smoker = len(bp_no_smoker)
ratio = n_no_smoker / n_smoker

solver = NormalIndPower()

power = solver.power(effect_size=d_welch,
                                nobs1=n_smoker,
                                alpha=alpha,
                                ratio=ratio,
                                alternative="two-sided")


print(f"Power (normalapproximation) ≈ {power:.3f}")

- Att power blir så lågt (0.074) innebär att sannolikheten att upptäcka verklig skillnad i blodtryck mellan rökare och icke-rökar är mycekt liten med det aktuella datamaterialet